1. Introduction
==

Decision trees are a powerful and popular machine learning technique. The basic concept is very similar to trees you may have seen commonly used to aid decision-making. Here's an example:


<img width="300" alt="creating a repo" src="https://drive.google.com/uc?export=view&id=1FYLn_lVbErtc7zNMneJ0yHlszXHSSEGt">

In the diagram above, we're deciding whether we should wrestle a bear that's in front of us. We're using various criteria to make our final decision, including the size of the bear, and whether escape is possible. Let's say we had a data set of people who survived bear encounters and the actions they took:

>```python
Bear name    Size    Escape possible?    Action
Yogi         Small   No                  Wrestle
Winnie       Small   Yes                 Wrestle
Baloo        Large   Yes                 Run away
Gentle Ben   Large   No                  Wrestle
```

If we wanted to optimize our chances of surviving a bear encounter, we could construct a decision tree to tell us what action to take.

As our data set gets larger though, this becomes less and less practical. What if we had 10000 rows and 10 variables? Would you want to look through all of the possibilities to construct a tree?

This is where the decision tree machine learning algorithm can help. It enables us to automatically construct a decision tree that tells us what outcomes we should predict in certain situations.

The decision tree algorithm is a supervised learning algorithm -- we first construct the tree with historical data, and then use it to predict an outcome. One of the major advantages of decision trees is that they can pick up nonlinear interactions between variables in the data that linear regression can't. In our bear wrestling example, a decision tree can pick up on the fact that you should only wrestle large bears when escape is impossible, whereas a linear regression would have had to weight both factors in the absence of the other.

We can use trees for classification or regression problems. In this lesson, we'll walk through the building blocks of making a decision tree automatically.

2. Overview of the Data Set
==

We'll be looking at individual income in the United States. The data is from the 1994 census, and contains information on an individual's marital status, age, type of work, and more. The target column, or what we want to predict, is whether individuals make less than or equal to 50k a year, or more than 50k a year.

You can download the data from the [University of California, Irvine's website](http://archive.ics.uci.edu/ml/datasets/Adult).


In [3]:
import pandas as pd

# Set index_col to False to avoid pandas thinking that the first column
# is row indexes (it's age)

income = pd.read_csv("income.csv", index_col=False)
income.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


3. Converting Categorical Variable
==

As we can see in our data, we have categorical variables such as **workclass** that have string values. Multiple individuals can share the same string value. The types of work include **State-gov**, **Self-emp-not-inc**, **Private**, and so on. Each of these strings is a label for a category. Another example of a column of categories is **sex**, where the options are **Male** and **Female**.

Before we get started with decision trees, we need to convert the categorical variables in our data set to numeric variables. This involves assigning a number to each category label, then converting all of the labels in a column to the corresponding numbers.

One strategy is to convert the columns to a [categorical](https://pandas-docs.github.io/pandas-docs-travis/categorical.html) type. Under this approach, pandas will display the labels as strings, but internally store them as numbers so we can do computations with them. The numbers aren't always compatible with other libraries like Scikit-learn, though, so it's easier to just do the conversion to numeric upfront.

We can use the [Categorical.from_array](http://pandas.pydata.org/pandas-docs/version/0.15.2/generated/pandas.core.categorical.Categorical.from_array.html) method from pandas to perform the conversion to numbers.

In [9]:
# Convert a single column from text categories to numbers
col = pd.Categorical(income["workclass"])
income["workclass"] = col.codes
income["workclass"].head(5)

0    7
1    6
2    4
3    4
4    4
Name: workclass, dtype: int8

In [11]:
for name in ["education", "marital_status", "occupation", "relationship", 
             "race", "sex", "native_country", "high_income"]:
    col = pd.Categorical(income[name])
    income[name] = col.codes

4. Splitting Data
==

A decision tree is made up of a series of nodes and branches. A node is where we split the data based on a variable, and a branch is one side of the split. Here's an example:

<img width="300" alt="creating a repo" src="https://drive.google.com/uc?export=view&id=1mQgPqrXt-l84ltP6EX9y1YXxDCaOCu7t">


In the diagram above, the node splits the data into two branches, **No** and **Yes**, based on whether the individual works in the private sector (the **workclass** column). We've mapped the value **Private** in the **workclass** column to the numeric code **4** (we can check this by comparing the values in the **workclass** column that used to have the label **Private** with the current values to see where they line up). So the **No** branch corresponds to **workclass != 4**, and the Yes branch corresponds to **workclass == 4**.

This is exactly how a decision tree works -- we keep splitting the data based on variables. As we do this, the tree accumulates more levels. The tree we made above is two levels deep because it has one split, and two "levels" of nodes.

The tree below is three levels deep.

<img width="300" alt="creating a repo" src="https://drive.google.com/uc?export=view&id=1rRro7BsYrLt36GMonxl3iOtrRH_QH4KK">

We added another split point to this tree "below", or after, our first split point. This made the tree three levels deep.



5. Creating Splits
==

Now think of rows of data flowing through a decision tree. In the diagram above, we can split the data set into two portions based on whether the individual works in the private sector or not.

In [13]:
# Enter your code here
private_incomes = income[income["workclass"] == 4]
public_incomes = income[income["workclass"] != 4]

print(private_incomes.shape)
print(public_incomes.shape)

(22696, 15)
(9865, 15)


6. Decision Trees as Flows of Data
==

When we performed the split, **9865** rows went to the left, where **workclass** does not equal **4**, and **22696** rows went to the right, where workclass equals **4**.

It's useful to think of a decision tree as a flow of rows of data. When we make a split, some rows will go to the right, and some will go to the left. As we build the tree deeper and deeper, each node will "receive" fewer and fewer rows.

Here's a look at the splits, and the number of rows that will exist at each node:

<img width="300" alt="creating a repo" src="https://drive.google.com/uc?export=view&id=1sO402qSORB1bfl3e-4l_f4e8c9WxgVDy">

7. Splitting Data to Make Predictions
==

The nodes at the bottom of the tree, where we decide to stop splitting, are called terminal nodes, or leaves. When we do our splits, we aren't doing them randomly; we have an objective. Our goal is to ensure that we can make a prediction on future data. In order to do this, all rows in each leaf must have only one value for our target column.

We're trying to predict the **high_income** column.
If **high_income** is **1**, it means that the person has an income higher than 50k a year.
If **high_income** is **0**, it means that they have an income less than or equal to 50k a year.

After constructing a tree using the **income** data, we'll want to make predictions. In order to do this, we'll take a new row and feed it through our decision tree.

First, we check whether the person works in the private sector. If they do, we'll check to see whether they're native to the US, and so on.

Eventually, we'll reach a leaf. The leaf will tell us what value we should predict for **high_income**.

In order to be able to make this prediction, all of the rows in a leaf should only have a single value for **high_income**. This means that leaves can't have both **0** and **1** values in the **high_income** column. Each leaf can only have rows with the same values for our target column. If this isn't the case, we won't be able to make effective predictions.

We'll need to continue splitting nodes until we get to a point where all of the rows in a node have the same value for **high_income**.

8. Overview of Data Set Entropy
==


Now that we have a high-level view of how decision trees work, let's explore the details and learn how to perform the splits.

We'll use a specific measure to figure out which variables we should split nodes on. Post-split, we'll have two data sets, each containing the rows from one branch of the split.

Because we're trying to reach the leaves having only **1s** or only **0s** in **high_income**, each split will need to get us closer to that goal.

When we split, we'll try to separate as many **0s** from **1s** in the high_income column as we can. In order to do this, we need a metric for how "together" the different values in the **high_income** column are.

Data scientists commonly use a metric called entropy for this purpose. Entropy refers to disorder. The more "mixed together" 1s and 0s are, the higher the entropy. A data set consisting entirely of 1s in the high_income column would have low entropy.

Entropy, which is not to be confused with entropy from physics, comes from [information theory](https://en.wikipedia.org/wiki/Information_theory). Information theory is based on probability and statistics, and deals with the transmission, processing, utilization, and extraction of information. A key concept in information theory is the notion of a bit of information. One bit of information is one [unit of information](https://en.wikipedia.org/wiki/Units_of_information).

We can represent a bit of information as a binary number because it either has the value **1** or **0**. Suppose there's an equal probability of tomorrow being sunny (**1**) or not sunny (**0**). If I tell you that it will be sunny, I've given you one bit of information.

We can also think of entropy in terms of information. If we flip a coin where both sides are heads, we know upfront that the result will be heads. We gain no new information by flipping the coin, so entropy is 0. On the other hand, if the coin has a heads side and a tails side, there's a 50% probability that it will land on either. Thus, flipping the coin gives us one bit of information -- which side the coin landed on.

Entropy can be much more complex, especially when we get to cases with more than two possible outcomes, or differential probabilities. A deep understanding of entropy isn't necessary for constructing decision trees, however. If you'd like, you can read more about entropy [at Wikipedia](https://en.wikipedia.org/wiki/Entropy).

The formula for entropy looks like this:

$$−\sum_{i=1}^{c}P(x_i)\log⁡_b{P(x_i)}$$

We iterate through each unique value in a single column (in this case, **high_income**), and assign it to i. We then compute the probability of that value occurring in the data (P(xi)). Next we do some multiplication, and sum all of the values together. b is the base of the logarithm. We commonly use the value **2** for this, but we can also set it to **10** or another value.

Let's say we have this data:

>```python
age    high_income
25     1
50     1
30     0
50     0
80     1
```

We could compute its entropy like this:

$$−\sum_{i=1}^{c}P(x_i)\log_b⁡ P(x_i)$$

$$−((2/5∗log_2⁡2/5)+(3/5∗log_2⁡3/5))$$

$$−(−0.5287712379549449+−0.44217935649972373)$$

$$.97$$

We get less than one "bit" of information -- only **.97** -- because there are slightly more **1s** in the sample data than **0s**. This means that if we were predicting a new value, we could guess that the answer is **1** and be right more often than wrong (because there's a **.6** probability of the answer being **1**). Due to this prior knowledge, we gain less than a full "bit" of information when we observe a new value.



In [15]:
import math
# We'll do the same calculation we did above, but in Python
# Passing in 2 as the second parameter to math.log will take a base 2 log
entropy = -(2/5 * math.log(2/5, 2) + 3/5 * math.log(3/5, 2))
print(entropy)

0.9709505944546686


In [17]:
prob_0 = income[income["high_income"] == 0].shape[0] / income.shape[0]
prob_1 = income[income["high_income"] == 1].shape[0] / income.shape[0]
income_entropy = -(prob_0 * math.log(prob_0, 2) + prob_1 * math.log(prob_1, 2))
income_entropy

0.7963839552022132

9. Information Gain 
==

We'll need a way to go from computing entropy to figuring out which variable to split on. We can do this using information gain, which tells us which split will reduce entropy the most.

Here's the formula for information gain:

$$IG(T,A)=Entropy(T)−\sum_{v\in A}\frac{\left |T_v \right |}{\left |T\right | }Entropy(T_v)$$


It may look complicated, but we'll break it down. We're computing information gain (IG) for a given target variable (T), as well as a given variable we want to split on (A).

To compute it, we first calculate the entropy for T. Then, for each unique value v in the variable A, we compute the number of rows in which A takes on the value v, and divide it by the total number of rows. Next, we multiply the results by the entropy of the rows where A is v. We add all of these subset entropies together, then subtract from the overall entropy to get information gain.

Here's an alternate explanation. We're finding the entropy of each set post-split, weighting it by the number of items in each split, then subtracting from the current entropy. If the result is positive, we've lowered entropy with our split. The higher the result is, the more we've lowered entropy.

One strategy for constructing trees is to create as many branches at each node as there are unique values for the variable we're splitting on. So if the variable has three or four values, we'd end up with three or four branches. This approach usually involves more complexity than it's worth and doesn't improve prediction accuracy, but it's worth knowing about.

To simplify the calculation of information gain and make splits simpler, we won't do it for each unique value. We'll find the median for the variable we're splitting on instead. Any rows where the value of the variable is below the median will go to the left branch, and the rest of the rows will go to the right branch. To compute information gain, we'll only have to compute entropies for two subsets.

Here's an example that uses the same data set we worked with earlier:

>```python
age    high_income
25     1
50     1
30     0
50     0
80     1
```

Let's say we wanted to split this data set based on age. First, we calculate the **median age**, which is **50**. Then, we assign any row with a value less than or equal to the median age the value 0 (in a new column named **split_age**), and the other rows 1.

>```python
age    high_income    split_age
25     1              0
50     1              0
30     0              0
50     0              0
80     1              1
```

Now we compute entropy:

$$IG(T,A)=Entropy(T)−\sum_{v\in A}\frac{\left | T_v \right |}{\left |T\right |}Entropy(T_v)$$

$$.97−(((4/5)∗−(1/2∗\log_21/2+1/2∗\log_21/2))+−(1/5∗(0∗\log_20+1∗\log_21)))$$

$$.97−((4/5)∗−(−.5+−.5))+(1/5∗−(0+1∗0))$$

$$.97−(4/5)$$

$$.169$$


We end up with .17, which means that we gain **.17** bits of information by splitting our data set on the **age** variable.

In [18]:
import numpy as np

def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    counts = np.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy

# Verify that our function matches our answer from earlier
entropy = calc_entropy([1,1,0,0,1])
print(entropy)

0.970950594455


In [19]:
information_gain = entropy - ((.8 * calc_entropy([1,1,0,0])) + (.2 * calc_entropy([1])))
print(information_gain)

0.170950594455


In [20]:
income_entropy = calc_entropy(income["high_income"])

median_age = income["age"].median()

left_split = income[income["age"] <= median_age]
right_split = income[income["age"] > median_age]

age_information_gain = income_entropy - ((left_split.shape[0] / income.shape[0]) * 
                                         calc_entropy(left_split["high_income"]) + 
                                         ((right_split.shape[0] / income.shape[0]) * 
                                          calc_entropy(right_split["high_income"])))
age_information_gain

0.047028661304691965

10. Finding the Best Split
==

Now that we know how to compute information gain, we can determine the best variable to split a node on. When we start our tree, we want to make an initial split. We'll find the variable to split on by calculating which split would have the highest information gain.



In [22]:
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    # Find the median of the column we're splitting
    column = data[split_name]
    median = column.median()
    
    # Make two subsets of the data, based on the median
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    # Return information gain
    return original_entropy - to_subtract

# Verify that our answer is the same as on the last screen
print(calc_information_gain(income, "age", "high_income"))

0.0470286613047


In [23]:
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]
information_gains = []
# Loop through and compute information gains
for col in columns:
    information_gain = calc_information_gain(income, col, "high_income")
    information_gains.append(information_gain)

# Find the name of the column with the highest gain
highest_gain_index = information_gains.index(max(information_gains))
highest_gain = columns[highest_gain_index]
highest_gain

'marital_status'

11. Next Steps
==

So far, we've been following the [ID3 algorithm](https://en.wikipedia.org/wiki/ID3_algorithm) to construct decision trees. There are other algorithms like [CART](ftp://public.dhe.ibm.com/software/analytics/spss/support/Stats/Docs/Statistics/Algorithms/13.0/TREE-CART.pdf) that use different measures for the split criterion. We'll learn more about these other algorithms in future missions.